<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: Huangjin Wang
#### Student ID: 32189222

Date: 22-08-2022


Environment: Python 3.9

Libraries used:
* os (for interacting with the operating system, included in Python 3.xxx package) 
* re (for regular expression, installed and imported) 
* pandas (for data manipulation) 
    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Review Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Whatever else](#latin) <br>
[5. Writing to an CSV File](#write) <br>
$\;\;\;\;$[5.1. Verification - using the sample files](#test_xml) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This assessment regards extracting data from semi-sctuctured text files. The dataset contained 500 `.txt` files which included various information about user reviews. In particular, each file only contains reviews' date, reviews' summary and reviews' text for a specific product id. The task is to extract all review texts, review summaries and latest review date for every product. The extracting process will use regular expression. Then, use pandas to create a dataframe to show latest review date, review summaries and review texts for each product id. Finally, output a csv file that contains those information.`os`library will be used to navigate throught folders, `re` library used to apply regular expressions, `pandas` library used to generate output.

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The only permitted packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **os:** to interact with the operating system, e.g. navigate through folders
* **re:** to define and use regular expressions
* **pandas:** to generate dataframe, and output csv file

In [1]:
import os
import re
import pandas as pd

-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Review Files <a class="anchor" name="examine"></a>

 </div>

Having examined the file content, the following observations were made:



* product id : consists of 10 characters that including upper case letters and numbers, e.g. `'B000AZJ268'` or `'1234567890'`, which always locate before the specific pattern `'<p>\n\n<h1>'`
* review text : there are two types of headings before review texts, e.g.`'review.text'` and `'REVIEW TEXT'`. and review text invovles of any alphanumerical characters ` 'a-z, A-Z, 0-9'` and special characters `!, ., -, ), `...
* review date : in the format of `'dd-mm-yyyy'`. e.g. `'14-06-2014'`
* review summary : there are two types of headings before review summaries, e.g.`'review.summary'` and `'REVIEW SUMMARY'`. and review text invovles of any alphanumerical characters ` 'a-z, A-Z, 0-9'` and special characters `!, ., -, ), `

Finally, we need to use pandas to create a dataframe to output a csv file

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

In this section, all the `.txt` files are parsed and processed. First of all, appropriate regular expressions are defined to extract desired information when reading the files. Then store these information for each pid

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

Defining correct regular expressions is crucial in extracting desired information from the text efficiently.

In [2]:
pid_pattern = r'\w{10}(?= <p>\n\n<h1>)'#reg ex pattern for id
date_pattern = r'[01][0-9]-[0-3][0-9]-[12][0-9][0-9][0-9]' #reg ex pattern for date
review_pattern = r"([Rr][Ee][Vv][Ii][Ee][Ww][\s.][Tt][Ee][Xx][Tt]\s+<\/h2>\n\n\s+<p>\s+)(.+)(<\/p>)" #reg ex pattern for reviews
summary_pattern = r'([Rr][Ee][Vv][Ii][Ee][Ww][\s.][Ss][Uu][Mm][Mm][Aa][Rr][Yy]\s+<\/h2>\n\n\s+<p>\s+)(.+)(\s+<\/p>\n\n\n<h2>)' #reg ex pattern for summaries

These patterns are used in the next step when reading the files.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Reading Files <a class="anchor" name="Read"></a>

In this step, all tweet files are read and parsed. In order to parse only the intended `.txt` files (and not other files in the folder such as Jupyter Notebook .ipynb_checkpoints), os.listdir will list all the directory in the txt folder, if it is not a directory, we will open it and do the text pre-processing. For each txt file readed, use regular expressions to extract pids, reviews, latest review date and review summaries.
* `pid_list` a list stores all pids extracted from the txt files
* `pid_date_dict` a dictionary to store pid and its latest review date, key is pid and value is latest review date
* `pid_review_dict` a dictionary to store pid and its review texts, key is pid and value is review texts
* `pid_sum_dict` a dictionary to store pid and its review summaries, key is pid and value is review summaries

A pre defined function to get the latest date, since the extracted date format likes `'mm-dd-yyyy'` instead of `'yyyy-mm-dd'`, we cannot directly use `max` function to get the latest date

In [3]:
def latest_day(d):
    ''' Compare the year, month and day for all extracted to dates get the latest date
    '''
    max_date = dates[0] # randomly select one to be the latest date
    for i in dates:
        date_list = max_date.split('-') 
        #Transfer string to a list, first element is day, second element is month, third element is year
        l1 = i.split('-')
        if l1[0].startswith('0'):
            #Day format like '06' needs to transfer to '6' for computation
            l1[0] = l1[0].replace('0','')

        if l1[1].startswith('0'):
            #Month format like '06' needs to transfer to '6' computation
            l1[1] = l1[1].replace('0','')

        if date_list[0].startswith('0'):
            #Day format like '06' needs to transfer to '6' for computation
            date_list[0] = date_list[0].replace('0','')

        if date_list[1].startswith('0'):
            #Month format like '06' needs to transfer to '6' computation
            date_list[1] = date_list[1].replace('0','')

        if int(l1[2])>int(date_list[2]):
            # If the year of this date is large than the max_date, this date will be the max_date
            max_date = i
            continue
        if int(l1[2])==int(date_list[2]) and int(l1[0])>int(date_list[0]):
            # If the month of this date is large than the max_date when year is the same, this date will be the max_date
            max_date = i
            continue
        if int(l1[2])==int(date_list[2]) and int(l1[0])==int(date_list[0]) and int(l1[1])>int(date_list[1]):
            # If the day of this date is large than the max_date when year and month is the same, this date will be the max_date
            max_date = i
            continue
    # After comparing all the dates, the latest date found
    return max_date

In [4]:
pid_list = []   
pid_date_dict = {}
pid_review_dict = {}
pid_sum_dict = {}

# Extract Product Id by using regular expression for each text file
path = 'D:/jupyter_notebook/32189222'
files = os.listdir(path)
for file in files:
    if not os.path.isdir(file): # if it is not a directory, read the file
        f = open(path + '/' + file, 'r')
        text_str = f.read()   # Read the text as a string
        # Extract pids
        pid = re.findall(pid_pattern,text_str)    # Apply regular expression
        pid_list.append(pid)                # Add extracted product id into the product id list
        #---------------------------------------
        # Extract the latest reveiw date
        dates = re.findall(date_pattern, text_str) # Apply regular expression
        latest_date = latest_day(dates)  # Use the function defined before to get the latest date
        pid_date_dict[str(pid)] = latest_date[3:5] + '/' +latest_date[:2] + '/' + latest_date[-4:] # change the date foramt to sample date format
        #---------------------------------------
        # Extract product reviews
        reviews = re.findall(review_pattern,text_str)
        for review in reviews: # store pids and corresponding reviews in a dict
            if str(pid)[2:-2] not in pid_review_dict.keys():
                pid_review_dict[str(pid)[2:-2]] = [review[1]]
            else:
                pid_review_dict[str(pid)[2:-2]].append(review[1])
        pid_review_dict[str(pid)[2:-2]] = str(pid_review_dict[str(pid)[2:-2]])# change the foramt to sample format
        #---------------------------------------
        # Extract review summaries
        summaries = re.findall(summary_pattern,text_str)
        for summary in summaries: # store pids and corresponding review summaries in a dict
            if str(pid)[2:-2] not in pid_sum_dict.keys():
                pid_sum_dict[str(pid)[2:-2]] = [summary[1]]
            else:
                pid_sum_dict[str(pid)[2:-2]].append(summary[1])
        pid_sum_dict[str(pid)[2:-2]] = str(pid_sum_dict[str(pid)[2:-2]]) # change the foramt to sample format
        f.close()

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.3. Creating a dataframe <a class="anchor" name="latin"></a>

Creating a dataframe by using pandas before we can output a csv file.

In [5]:
data_dict = {}
data_dict['PID'] = pid_review_dict.keys()
data_dict['product_review'] = pid_review_dict.values()
data_dict['review_summary'] = pid_sum_dict.values()
data_dict['latest_review_date'] = pid_date_dict.values()
df = pd.DataFrame(data_dict)
df

,PID,product_review,review_summary,latest_review_date
0,B00004Z8BC,"['If you are replacing your UPS battery, go w/...","['Go With OEM', 'APC UPS replacement battery',...",14/06/2014
1,B00066ISFM,"[""Bought this based on other reviews and some ...","['Good protection in a small package', 'Works ...",18/06/2014
2,B005L38SGY,['This is as it is described a gaming headset....,['Sounds good but... However it is great for g...,30/03/2014
3,B005MI8NJK,"[""While the sound quality of these buds is pre...","['OK but I expect more at this price level', '...",29/03/2013
4,B005O4FYVC,"[""These headphones adjust to fit even my two y...","['Great Headphones Even For Kids', 'cheap made...",02/04/2014
...,...,...,...,...
495,B0056HNRPW,['As a beginner who just has taken 1 class on ...,"['worked well', 'Syba 50 piece tool kit', 'Awe...",21/01/2014
496,B005BUWQ5G,"[""Well, the clip on these are cheap plastic an...",['Well...at least they are good spares and goo...,17/03/2014
497,B005DOMNK8,['The cameras are easy to setup and have a sta...,"['Good enough, but not excellent', 'good', 'Ge...",14/06/2014
498,B005GMWNY8,"[""Smooth pan and tilt; doesn't have all the be...","['Best $70 fluid head tripod??', 'Broke First ...",27/06/2014


-------------------------------------

<div class="alert alert-block alert-warning"> 

## 5.  Writing to an CSV File <a class="anchor" name="write"></a>

</div>

Output a csv file and remove index to follow the sample output structrue.

In [6]:
df.to_csv('32189222.csv',index = False)

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Verification of the Generated CSV File <a class="anchor" name="test_xml"></a>

Read the output file, and check if structure is same as sample output file and check the lenth

In [7]:
df1 = pd.read_csv('32189222.csv')
df1

,PID,product_review,review_summary,latest_review_date
0,B00004Z8BC,"['If you are replacing your UPS battery, go w/...","['Go With OEM', 'APC UPS replacement battery',...",14/06/2014
1,B00066ISFM,"[""Bought this based on other reviews and some ...","['Good protection in a small package', 'Works ...",18/06/2014
2,B005L38SGY,['This is as it is described a gaming headset....,['Sounds good but... However it is great for g...,30/03/2014
3,B005MI8NJK,"[""While the sound quality of these buds is pre...","['OK but I expect more at this price level', '...",29/03/2013
4,B005O4FYVC,"[""These headphones adjust to fit even my two y...","['Great Headphones Even For Kids', 'cheap made...",02/04/2014
...,...,...,...,...
495,B0056HNRPW,['As a beginner who just has taken 1 class on ...,"['worked well', 'Syba 50 piece tool kit', 'Awe...",21/01/2014
496,B005BUWQ5G,"[""Well, the clip on these are cheap plastic an...",['Well...at least they are good spares and goo...,17/03/2014
497,B005DOMNK8,['The cameras are easy to setup and have a sta...,"['Good enough, but not excellent', 'good', 'Ge...",14/06/2014
498,B005GMWNY8,"[""Smooth pan and tilt; doesn't have all the be...","['Best $70 fluid head tripod??', 'Broke First ...",27/06/2014


-------------------------------------

<div class="alert alert-block alert-warning"> 

## 6. Summary <a class="anchor" name="summary"></a>

</div>

In conclusion, we used reluar expressions to extract pids and their review texts, latest review date and review summaries for 500 txt files. os library used to find txt file, re library used to apply regular expressions to the text, and pandas library used to work with dataframe and output a csv file.

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 7. References <a class="anchor" name="Ref"></a>

</div>

[1 ]<a class="anchor" name="ref-2"></a> Regular Expression Language - Quick Reference,https://docs.microsoft.com/en-us/dotnet/standard/base-types/regular-expression-language-quick-reference, Accessed 26/08/2022

[2 ]<a class="anchor" name="ref-2"></a> pandas.DataFrame, https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html, Accessed 27/08/2022

[3 ]<a class="anchor" name="ref-2"></a> Open All the Files in a Directory in Python,https://www.delftstack.com/howto/python/python-open-all-files-in-directory/, Accessed 26/08/2022

## --------------------------------------------------------------------------------------------------------------------------